In [1]:
#!pip install transformers
#!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
import os
import re
import glob
from sklearn.preprocessing import MultiLabelBinarizer

import torch.nn as nn
import xml.etree.ElementTree as ET
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import random
from os import listdir
from os.path import isfile, join
from sklearn import preprocessing
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from transformers import Trainer, TrainingArguments
from datasets import load_metric
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification
import ast
# from argparse import ArgumentParser
# parser = ArgumentParser()
# parser.add_argument("-multi_lb_classi", default=1, type=int, help="whether train a multi-lable classifier or individual classifiers for each class")
# opt = parser.parse_args()
multi_lb_classi = 0
docs = os.listdir("./scisumm-2018/Training")
train_e = int(0.8*len(docs))
train_docs = docs[:train_e]
test_docs = docs[train_e:]
# test_docs  = os.listdir("./scisumm-2018/Test")
stop_words = set(stopwords.words('english'))

def preprocess(example_sent):
    global stop_words
    word_tokens = word_tokenize(example_sent.lower())
    filtered_sentence = [w for w in word_tokens if not w in stop_words and w.isalpha()]
    new = " " 
    a = new.join(filtered_sentence)
    return a



In [2]:
# import nltk
# nltk.download('stopwords')

In [3]:
# nltk.download('punkt')

In [4]:
def get_dataset(files, folder):
    data_lis = []
    labels = []
    sid_lis = []
    ssid_lis = []
    for z,f in enumerate(files):
        data = None
        ann = None
        a_folder = folder+f+"/annotation/"
        file = os.listdir(a_folder)[0]

        ann = a_folder+file
        with open(ann,"r") as f:
            data = f.read()
#         print(f)
            facet = re.findall("Discourse Facet:\s+([^|]*)", data)
            for f in facet:
                if '[' in f:
                    f = ast.literal_eval(f)
                    f_lis = []
                    for a in f:
                        t_lb = a.strip().lower()
                        if 'results' in t_lb:
                            f_lis.append(t_lb.replace('results','result'))
                        else:
                            f_lis.append(t_lb)
                    labels.append(f_lis)
                        
                else:
                    t_lb = f.strip().lower()
                    if 'results' in t_lb:
                        t_lb = t_lb.replace('results','result')
                    labels.append([t_lb])

            
            ref_text = re.findall("Reference Text:\s+([^|]*)", data)
            pattern = r'\<.*?\>'
            pattern2 = r'\(.*?\)'
            
            
            for ref in ref_text:
                ssid_per_citant = []
                sid_per_citant = []
                temp = re.findall(r'\<S.*?\>',ref)
                sids = re.findall(r'<S sid =.*?\s',ref)
                ssids = re.findall(r'ssid =.*?>',ref)
                for i,sid in enumerate(sids):
                    sid = sid.split('=')[1]
                    
                    try:
                        sid_per_citant.append(int(sid.replace('"','').strip()))
                    except:
#                         print(sid)
                        sid = sid.split('>')[0]
                        sid_per_citant.append(int(sid.replace('"','').strip()))
                    try:
                        ssid = ssids[i].split('=')[1].replace('>',"")
                    except:
                        ssid = sid
                    ssid_per_citant.append(int(ssid.replace('"','').strip()))
                sid_lis.append(sid_per_citant)
                ssid_lis.append(ssid_per_citant)
                ref = re.sub(pattern2,'',re.sub(pattern, '', ref))
                ref = preprocess(ref)
                data_lis.append(ref)

            
    return data_lis, sid_lis, ssid_lis,labels


In [5]:
# def get_test_dataset(test_docs):
#     '''
#     gives citance and labels
#     '''
#     data_lis = []
#     labels = []
#     for z,f in enumerate(files):
#         df = folder+f+"/annotation/"+f+".csv"
#         data_lis.append()

# test_data,test_labels = get_dataset(test_docs[:2],folder='scisumm-2018/Test/')



In [6]:
train_data,train_sid, train_ssid, train_labels = get_dataset(train_docs,"scisumm-2018/Training/")
test_data, test_sid, test_ssid, test_labels = get_dataset(test_docs,"scisumm-2018/Training/")

In [7]:
len(train_data), len(train_labels), len(train_sid), len(train_ssid)

(590, 590, 590, 590)

In [8]:
len(test_data), len(test_labels)

(163, 163)

In [9]:
train_ssid

[[15],
 [2, 15],
 [21, 7],
 [25],
 [34, 67],
 [25, 26, 27],
 [2, 3, 21],
 [7, 10],
 [10, 44],
 [10, 13],
 [21],
 [17, 21],
 [21],
 [21, 9, 6, 65, 76],
 [26, 29, 30],
 [5],
 [6],
 [9],
 [1],
 [9],
 [3],
 [3],
 [15],
 [4],
 [22],
 [100],
 [1],
 [3],
 [6],
 [9],
 [1],
 [1],
 [104],
 [104],
 [27, 34, 2],
 [16],
 [97],
 [20, 21],
 [97],
 [6],
 [3],
 [1, 2, 10],
 [52],
 [3],
 [26],
 [22],
 [5, 6, 7],
 [3],
 [3, 4],
 [2, 6],
 [6, 1, 2, 52],
 [28],
 [8],
 [1],
 [17],
 [9],
 [17],
 [16, 17],
 [63],
 [63],
 [68],
 [63],
 [6],
 [24],
 [24, 82, 101],
 [11],
 [21, 22, 27, 8],
 [38],
 [4, 5],
 [16],
 [1],
 [94],
 [11],
 [11],
 [11],
 [46],
 [21],
 [46],
 [70, 72],
 [0],
 [24],
 [0],
 [70, 72, 24],
 [20],
 [94],
 [94],
 [33, 34],
 [21],
 [12],
 [21, 22, 23],
 [4, 5],
 [3],
 [8, 9, 10],
 [39, 40, 22],
 [23, 4],
 [4],
 [11, 14],
 [2, 22, 23, 24],
 [22, 23, 24],
 [23, 24],
 [23, 2, 3],
 [24],
 [12],
 [50],
 [130],
 [2],
 [46],
 [16],
 [23],
 [58],
 [129],
 [50],
 [33],
 [132],
 [37, 38],
 [1, 10],
 [37,

## multi-label classification

In [10]:
# le = preprocessing.LabelEncoder()
# le.fit(train_labels)
# train_labels = le.transform(train_labels)
# # val_labels = le.transform(val_labels)
# test_labels = le.transform(test_labels)

In [11]:
train_labels

[['method_citation'],
 ['hypothesis_citation', 'method_citation'],
 ['method_citation'],
 ['result_citation'],
 ['method_citation', 'result_citation'],
 ['implication_citation', 'method_citation'],
 ['hypothesis_citation', 'method_citation'],
 ['implication_citation'],
 ['method_citation'],
 ['implication_citation'],
 ['hypothesis_citation', 'aim_citation'],
 ['hypothesis_citation', 'aim_citation'],
 ['hypothesis_citation', 'aim_citation'],
 ['method_citation', 'result_citation'],
 ['method_citation'],
 ['method_citation'],
 ['method_citation'],
 ['method_citation'],
 ['method_citation'],
 ['method_citation'],
 ['method_citation'],
 ['method_citation'],
 ['method_citation'],
 ['result_citation'],
 ['result_citation'],
 ['method_citation'],
 ['method_citation'],
 ['method_citation'],
 ['result_citation'],
 ['result_citation'],
 ['method_citation'],
 ['method_citation'],
 ['method_citation'],
 ['aim_citation'],
 ['method_citation'],
 ['result_citation'],
 ['method_citation'],
 ['method_c

In [12]:
len(test_data), len(test_labels)

(163, 163)

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
train_labels_lb = mlb.fit_transform(train_labels)
test_labels_lb = mlb.transform(test_labels)

/home/avani.gupta/miniconda3/envs/nlp/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['method citation'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))


In [14]:
len(train_labels_lb)

590

In [15]:
len(test_labels_lb)

163

In [16]:
mlb.classes_

array(['aim_citation', 'hypothesis_citation', 'implication_citation',
       'method_citation', 'result_citation'], dtype=object)

In [17]:
len(train_data), len(train_labels_lb)

(590, 590)

In [18]:
def calc_metrics(pred_lis, gt_lis):
    acc = accuracy_score(pred_lis, gt_lis)
    recall = recall_score(pred_lis, gt_lis,average='micro')
    prec = precision_score(pred_lis, gt_lis,average='micro')
    f1 = f1_score(pred_lis, gt_lis,average='micro')
    print("metrics obtained in test: accuracy {} recall {}, precision {}, f1-score {}".format(acc,recall,prec,f1))
    
# def get_integer_mapping(le):
#     '''
#     Return a dict mapping labels to their integer values
#     from an SKlearn LabelEncoder
#     le = a fitted SKlearn LabelEncoder
#     '''
#     res = {}
#     for cl in le.classes_:
#         res.update({le.transform([cl])[0]:cl})

#     return res


# mapping = get_integer_mapping(mlb)
# mapping

In [19]:
# val_data,val_labels = get_dataset(val_docs,folder='From-ScisummNet-2019')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_data,return_tensors="pt",max_length=512,padding='max_length',truncation=True)
# val_encodings = tokenizer(val_data,return_tensors="pt",max_length=512,padding='max_length',truncation=True)
test_encodings = tokenizer(test_data,return_tensors="pt",max_length=512,padding='max_length',truncation=True)



In [20]:
test_encodings_orig = tokenizer(test_data,return_tensors="pt",max_length=512,padding='max_length',truncation=True)

In [21]:
len(train_encodings),len(train_labels),len(train_data)

(3, 590, 590)

In [22]:
train_encodings['input_ids'].shape, len(train_sid)

(torch.Size([590, 512]), 590)

In [23]:
train_sid

[[15],
 [2, 15],
 [21, 37],
 [152],
 [124, 194],
 [115, 116, 117],
 [2, 3, 21],
 [7, 10],
 [10, 83],
 [10, 13],
 [21],
 [17, 21],
 [21],
 [21, 39, 45, 192, 203],
 [116, 119, 120],
 [5],
 [6],
 [9],
 [48],
 [9],
 [3],
 [3],
 [15],
 [99],
 [22],
 [239],
 [52],
 [112],
 [6],
 [9],
 [52],
 [52],
 [243],
 [243],
 [27, 85, 97],
 [155],
 [236],
 [20, 21],
 [236],
 [6],
 [3],
 [17, 18, 113],
 [68],
 [3],
 [42],
 [125],
 [5, 6, 7],
 [3],
 [3, 4],
 [2, 6],
 [6, 17, 18, 68],
 [44],
 [8],
 [1],
 [17],
 [45],
 [17],
 [16, 17],
 [270],
 [270],
 [275],
 [270],
 [6],
 [24],
 [24, 289, 308],
 [11],
 [21, 22, 53, 141],
 [64],
 [116, 117],
 [128],
 [312],
 [301],
 [11],
 [11],
 [11],
 [72],
 [21],
 [72],
 [277, 279],
 [0],
 [24],
 [0],
 [277, 279, 24],
 [153],
 [301],
 [301],
 [166, 167],
 [154],
 [12],
 [21, 22, 23],
 [205, 206],
 [3],
 [186, 187, 188],
 [39, 40, 78],
 [23, 205],
 [205],
 [11, 14],
 [203, 22, 23, 24],
 [22, 23, 24],
 [23, 24],
 [23, 141, 142],
 [24],
 [29],
 [123],
 [203],
 [62],
 [119]

In [24]:
train_sid_= np.array(train_sid)
train_sid_ = np.expand_dims(train_sid_, axis=1)

test_sid_= np.array(test_sid)
test_sid_ = np.expand_dims(test_sid_, axis=1)

/home/avani.gupta/miniconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.
/home/avani.gupta/miniconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


In [25]:
train_sid_.shape

(590, 1)

In [26]:
train_data_all = np.concatenate((train_encodings['input_ids'],train_sid_),axis=1)
test_data_all = np.concatenate((test_encodings['input_ids'],test_sid_),axis=1)

In [27]:
len(train_encodings['input_ids'])

590

In [28]:
train_labels_lb

array([[0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [1, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0]])

In [29]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier

def classify(classifier):
    classifier.fit(train_encodings['input_ids'], train_labels_lb)
    pred_lis = classifier.predict(test_encodings['input_ids'])
    calc_metrics(pred_lis, test_labels_lb)
classifier = BinaryRelevance(GaussianNB())
# classify(classifier)

In [30]:
from skmultilearn.problem_transform import ClassifierChain

classifier = ClassifierChain(GaussianNB())
# classify(classifier)

In [31]:
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB
classifier = LabelPowerset(GaussianNB())
# classify(classifier)

## BERT

In [32]:
def split_two_labels(data, labels):
    '''
    split two labels in 2 data-points
    '''
    labels_new = []
    data_new = []
    for i,lb in enumerate(labels):
        if type(lb) == list: #needs to be split
            for l in lb:
                l = l.lower()
                if '_' in l:
                    l = l.split('_')[0]
                else:
                    l = l.split()[0]
                if l=='results':
                    l = 'result'
                labels_new.append(l)
                data_new.append(data[i])
        else:
            l = lb.lower()
            if '_' in l:
                    l = l.split('_')[0]
            else:
                l = l.split()[0]
            if l=='results':
                l = 'result'
            labels_new.append(l)
            data_new.append(data[i])
    return data_new, labels_new

train_data, train_labels = split_two_labels(train_data, train_labels)
test_data, test_labels  = split_two_labels(test_data, test_labels )

In [33]:
le = preprocessing.LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
# val_labels = le.transform(val_labels)
test_labels = le.transform(test_labels)

In [34]:
len(train_data), len(train_labels)

(639, 639)

In [35]:
# val_data,val_labels = get_dataset(val_docs,folder='From-ScisummNet-2019')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_data,return_tensors="pt",max_length=512,padding='max_length',truncation=True)
# val_encodings = tokenizer(val_data,return_tensors="pt",max_length=512,padding='max_length',truncation=True)
test_encodings = tokenizer(test_data,return_tensors="pt",max_length=512,padding='max_length',truncation=True)



In [36]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, torch.from_numpy(train_labels))
test_dataset = IMDbDataset(test_encodings, torch.from_numpy(test_labels))


In [37]:
def get_integer_mapping(le):
    '''
    Return a dict mapping labels to their integer values
    from an SKlearn LabelEncoder
    le = a fitted SKlearn LabelEncoder
    '''
    res = {}
    for cl in le.classes_:
        res.update({le.transform([cl])[0]:cl})

    return res


mapping = get_integer_mapping(le)
mapping

{0: 'aim', 1: 'hypothesis', 2: 'implication', 3: 'method', 4: 'result'}

In [38]:
inv_mapping =  dict([(value, key) for key, value in mapping.items()])
inv_mapping

{'aim': 0, 'hypothesis': 1, 'implication': 2, 'method': 3, 'result': 4}

In [39]:
def train_sep_class_classifiers():
    for cl in mapping:
#     cl = 2
        training_args = TrainingArguments(
        output_dir='/ssd_scratch/cvit/avani.gupta/results'+str(cl),          # output directory
        num_train_epochs=3,              # total number of training epochs
        per_device_train_batch_size=1,  # batch size per device during training
        per_device_eval_batch_size=1,   # batch size for evaluation
        warmup_steps=500,               # number of warmup steps for learning rate scheduler
        weight_decay=0.01,              # strength of weight decay
        logging_dir='/ssd_scratch/cvit/avani.gupta/logs'+str(cl),            # directory for storing logs
        logging_steps=20000,
        save_steps = 30000
        )
        print("for class: ",mapping[cl])
        train_ind = np.where(train_labels==cl)[0].astype(int)
        new_train_labels = np.zeros(train_labels.shape).astype(int)
        new_train_labels[train_ind] = 1

        # val_ind = np.where(val_labels==cl)[0].astype(int)
        test_ind = np.where(test_labels==cl)[0].astype(int)
        new_test_labels = np.zeros(test_labels.shape).astype(int)
        new_test_labels[test_ind] = 1


        train_dataset = IMDbDataset(train_encodings, torch.from_numpy(new_train_labels))
        test_dataset = IMDbDataset(test_encodings, torch.from_numpy(new_test_labels))    


        model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        model.to(device)
        #model = nn.DataParallel(model,device_ids=[0,1,2,3])

        trainer = Trainer(
            model=model,                         # the instantiated 🤗 Transformers model to be trained
            args=training_args,                  # training arguments, defined above
            train_dataset=train_dataset         # training dataset
        )
        trainer.train()
        # eval
        model.eval()
        torch.save(model, "bert_single_lb_classi"+str(cl)+".pth")
        val_dataloader = DataLoader(test_dataset, batch_size=8)
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        pred_lis = []
        gt_lis = []

        for batch in val_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            pred_lis.extend(predictions.cpu().numpy())
            gt_lis.extend(batch["labels"].cpu().numpy())

        calc_metrics(pred_lis, gt_lis)


# train_sep_class_classifiers()


In [40]:
mapping  =  {0: 'aim', 1: 'hypothesis', 2: 'implication', 3: 'method', 4: 'result'}

In [41]:
test_encodings_orig

{'input_ids': tensor([[  101, 22073,  8454,  ...,     0,     0,     0],
        [  101,  6415,  4590,  ...,     0,     0,     0],
        [  101,  2795,  2358,  ...,     0,     0,     0],
        ...,
        [  101,  3120,  2773,  ...,     0,     0,     0],
        [  101,  6611,  7504,  ...,     0,     0,     0],
        [  101,  3259,  6235,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [42]:
input_ids_lis = test_encodings_orig['input_ids']
token_type_ids_lis = test_encodings_orig['token_type_ids']
attention_mask_lis = test_encodings_orig['attention_mask']

In [43]:
preds = {}
for cl in mapping:
    preds[cl] = []
    device = 'cuda'
    model = torch.load("bert_single_lb_classi"+str(cl)+".pth").eval()
    for i in range(len(input_ids_lis)):
        model.to(device)
        outputs = model(input_ids = input_ids_lis[i].unsqueeze(0).to(device),token_type_ids=token_type_ids_lis[i].unsqueeze(0).to(device),attention_mask=attention_mask_lis[i].unsqueeze(0).to(device))
        preds[cl].append(torch.argmax(outputs.logits.cpu().detach(), dim=-1).numpy()[0])

test_pred_lb= np.concatenate((np.expand_dims(np.array(preds[0]),1),np.expand_dims(np.array(preds[1]),1),np.expand_dims(np.array(preds[2]),1),np.expand_dims(np.array(preds[3]),1),np.expand_dims(np.array(preds[4]),1)),axis=1)

In [44]:
y = np.where((test_pred_lb == [0, 0, 0, 0, 0]).all(axis = 1))
test_pred_lb[y]  = [0,0,0,1,0]
calc_metrics(test_pred_lb,test_labels_lb)

metrics obtained in test: accuracy 0.6073619631901841 recall 0.6590909090909091, precision 0.7341772151898734, f1-score 0.6946107784431138


In [45]:

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, torch.from_numpy(train_labels_lb))
test_dataset = IMDbDataset(test_encodings, torch.from_numpy(test_labels_lb))

In [46]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [47]:
from transformers import BertModel

class Net(nn.Module):
    def __init__(self,n_classes=5):
        super(Net,self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased',return_dict=True)
        self.classifier=nn.Linear(self.bert.config.hidden_size,n_classes) 
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,input_ids, token_type_ids, attention_mask):
        output = self.bert(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        output = self.classifier(output.pooler_output)
        output = self.sigmoid(output)

        return output

In [48]:
model = Net()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [49]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [50]:
train_dataloader = DataLoader(train_dataset, batch_size=4)
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

Net(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        

In [70]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
loss = nn.BCELoss()
model.train()
def train():
    for epoch in range(num_epochs):
        ep_loss = 0
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(batch['input_ids'],batch['token_type_ids'], batch['attention_mask'])
            l = loss(outputs,batch['labels'].float())
            l.backward()
            ep_loss += l.item()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
        print("epoch {} loss {}".format(epoch,ep_loss))
        
    torch.save(model,'multi_lb_bert.pth')
train()

  0%|          | 0/1770 [00:00<?, ?it/s]

/home/avani.gupta/miniconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/avani.gupta/miniconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


epoch 0 loss 197.2358829677105
epoch 1 loss 197.27312152087688
epoch 2 loss 197.20827310532331


In [72]:
model = torch.load('multi_lb_bert.pth')

In [87]:
preds = []
device = 'cuda'
model.eval()
for i in range(len(input_ids_lis)):
    model.to(device)
    outputs = model(input_ids = input_ids_lis[i].unsqueeze(0).to(device),token_type_ids=token_type_ids_lis[i].unsqueeze(0).to(device),attention_mask=attention_mask_lis[i].unsqueeze(0).to(device))
    preds.append(outputs.cpu().detach().numpy()[0])

# test_pred_lb= np.concatenate((np.expand_dims(np.array(preds[0]),1),np.expand_dims(np.array(preds[1]),1),np.expand_dims(np.array(preds[2]),1),np.expand_dims(np.array(preds[3]),1),np.expand_dims(np.array(preds[4]),1)),axis=1)

In [88]:
preds = np.array(preds)

In [89]:
preds[preds >= 0.6] = 1
preds[preds < 0.6] = 0

In [92]:
preds = preds.astype(int)

In [93]:
calc_metrics(preds,test_labels_lb)

metrics obtained in test: accuracy 0.6441717791411042 recall 0.6748466257668712, precision 0.6962025316455697, f1-score 0.6853582554517134
